<h1 style="text-align: center;">Máquinas de Turing</h1>

<div style="text-align: justify;">
<p>
    Una máquina de Turing es una máquina hipotética ideada por el matemático Alan Turing en 1936. A pesar de su simplicidad, la máquina puede simular CUALQUIER algoritmo informático, ¡por complicado que sea!</p> 
<p>
    Consiste en una cinta de longitud infinita que actúa como la memoria de un ordenador normal o cualquier otra forma de almacenamiento de datos. Las casillas de la cinta suelen estar en blanco al principio y pueden escribirse con símbolos. En este caso, la máquina sólo puede procesar los símbolos 0 y 1 y « » (en blanco), por lo que se dice que es una máquina de Turing de 3 símbolos. En cualquier momento, la máquina tiene una cabeza que se coloca sobre uno de los cuadrados de la cinta. Con este cabezal, la máquina puede realizar tres operaciones muy básicas:</p>
<ul>
    <li>Leer el símbolo de la casilla situada bajo el cabezal.</li>
    <li>Editar el símbolo escribiendo uno nuevo o borrándolo.</li>
    <li>Desplazar la cinta una casilla a la izquierda o a la derecha para que la máquina pueda leer y editar el símbolo de una casilla vecina.</li>   
</ul>
</div>
<div style="text-align: center;">
    <img src="https://www.cl.cam.ac.uk/projects/raspberrypi/tutorials/turing-machine/example_turing_tape.jpg" alt="Texto alternativo"/>
</div>

Con los símbolos impresos en la cinta, el siguiente programa sencillo convierte los 1s en 0s y viceversa. Esto se conoce como inversión de bits y puede hacerse pasando una funcion de transicion a la máquina de Turing. La máquina leerá primero el símbolo situado bajo el cabezal, escribirá un nuevo símbolo en consecuencia y moverá la cinta a la izquierda, antes de repetir de nuevo la secuencia.

In [21]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as am
from IPython.display import HTML, display

class MT:
    def __init__(self, cadena='', vacio=' ', estado_inicial='q0', estados_finales=None, funcion_transicion=None):
        self.cinta = list(cadena)
        self.vacio = vacio
        self.posicion_cabeza = 0
        self.estado_actual = estado_inicial
        self.estados_finales = estados_finales if estados_finales is not None else set()
        self.funcion_transicion = funcion_transicion if funcion_transicion is not None else {}
        if not self.cinta:
            self.cinta = [vacio]

    def paso(self):
        simbolo_actual = self.cinta[self.posicion_cabeza]
        
        if (self.estado_actual, simbolo_actual) in self.funcion_transicion:
            estado_siguiente, escribir_simbolo, direccion_movimiento = self.funcion_transicion[(self.estado_actual, simbolo_actual)]
            
            self.cinta[self.posicion_cabeza] = escribir_simbolo
            
            if direccion_movimiento == 'R':
                self.posicion_cabeza += 1
            elif direccion_movimiento == 'L':
                self.posicion_cabeza -= 1
                
            if self.posicion_cabeza < 0:
                self.cinta.insert(0, self.vacio)
                self.posicion_cabeza = 0
            if self.posicion_cabeza >= len(self.cinta):
                self.cinta.append(self.vacio)
            
            self.estado_actual = estado_siguiente
            return True
        else:
            return False

    def ejecutar(self, max_pasos=1000):
        pasos = []
        contador_pasos = 0
        while self.estado_actual not in self.estados_finales and contador_pasos < max_pasos:
            pasos.append((list(self.cinta), self.posicion_cabeza, self.estado_actual))
            if not self.paso():
                break
            contador_pasos += 1
        pasos.append((list(self.cinta), self.posicion_cabeza, self.estado_actual))  
        return pasos


funcion_transicion = {
    ('q0', '0'): ('q0', '1', 'R'),
    ('q0', '1'): ('q0', '0', 'R'),
    ('q0', ' '): ('qf', ' ', 'N'),
    ('q1', '0'): ('q1', '1', 'R'),
    ('q1', '1'): ('q1', '0', 'R'),
    ('q1', ' '): ('q1', '', 'N')
}

mt = MT(cadena='110100110101', estado_inicial='q0', estados_finales={'qf'}, funcion_transicion=funcion_transicion)
pasos = mt.ejecutar()

def animar(i):
    cinta, posicion_cabeza, estado_actual = pasos[i]
    cinta_str = ''.join(cinta).replace(' ', '_') 
    cinta_display = list(cinta_str)
    
    ax.clear()
    for idx, simbolo in enumerate(cinta_display):
        rect = patches.Rectangle((idx, 0), 1, 2, fill=None, edgecolor='black')
        ax.add_patch(rect)
        ax.text(idx + 0.5, 1, simbolo, ha='center', va='center', fontsize=16)
    cabeza_rect = patches.Rectangle((posicion_cabeza, 0), 1, 2, fill=None, edgecolor='r', linewidth=2.5)
    ax.add_patch(cabeza_rect)
    ax.set_xlim(-1, len(cinta_display) + 1)
    ax.set_ylim(-1, 2)
    ax.axis('off')

fig, ax = plt.subplots(figsize=(10, 2))
ani = am.FuncAnimation(fig, animar, frames=len(pasos), interval=500, repeat=False)
ani_html = ani.to_jshtml()
plt.close()
display(HTML(ani_html))